Uvozimo potrebne knjižnice za manipulacijo s podatki:

In [1]:
import random
import math
import matplotlib.pylab as plt

Najprej določimo točke v kvadratu velikosti 1x1. Točke so določena z dvema koordinatama (x,y), kjer x označuje premik po vodoravni stranici, y pa premik po navpični stranici. 

Oglišča kvadrata so (0,0), (0,1), (1,0), (1,1).

In [2]:
def kvadrat_tock(st_tock):
    slovar_tock = {}
    for i in range(st_tock):
        x = random.random()
        y = random.random()
        slovar_tock['tocka_' + str(i)] = (x,y)
    return slovar_tock

tocke = kvadrat_tock(8)
primer_1 = {'tocka_0': (0.1, 0.1), 'tocka_1': (0.2, 0.1),  'tocka_2': (0.1,
0.2),  'tocka_3': (0.2, 0.2)}

primer_2 = {'tocka_0': (0.1, 0.1), 'tocka_1': (0.2, 0.1), 'tocka_2': (0.3, 0.1),  'tocka_3': (0.4, 0.1),
           'tocka_4': (0.5, 0.1), 'tocka_5': (0.6, 0.1), 'tocka_6': (0.7, 0.1),  'tocka_7': (0.8, 0.1),
           'tocka_8': (0.9, 0.1), 'tocka_9': (1, 0.1)}

primer_3 = {'tocka_0': (0, 0), 'tocka_1': (0, 1/3), 'tocka_2': (0, 2/3),  'tocka_3': (0, 1),
           'tocka_4': (1/3, 0), 'tocka_5': (1/3, 1/3), 'tocka_6': (1/3, 2/3),  'tocka_7': (1/3, 1),
           'tocka_8': (2/3, 0), 'tocka_9': (2/3, 1/3), 'tocka_10': (2/3, 2/3),  'tocka_11': (2/3, 1)}

Razdalje med točkami izračunamo s pomočjo Pitagorovega izreka.

In [3]:
def razdalje(slovar):
    slovar_razdalj = {}
    #a^2 + b^2 = c^2
    for key1 in slovar.keys():
        seznam_razdalj = []
        # for key in slovar.keys()
        for key2 in slovar.keys():
            if key1 == key2:
                continue
            delta_x = slovar[key1][0] - slovar[key2][0]
            delta_y = slovar[key1][1] - slovar[key2][1]
            razdalja = sqrt(delta_x^2 + delta_y^2)
            seznam_razdalj.append((key2,razdalja))
        slovar_razdalj[key1] = seznam_razdalj
    return(slovar_razdalj)

#razdalje(tocke)

Posamezna razdalja nam predstavlja preferenco (manjša kot je razdalja, večja je preferenca). Le te uredimo po velikosti, od najmanjše do največje, za vsako točko:

In [4]:
def preference(slovar_razdalj):
    for key in slovar_razdalj.keys():
        sorted_by_second = sorted(slovar_razdalj[key], key=lambda tup: tup[1])
        slovar_razdalj[key] = sorted_by_second
    return(slovar_razdalj)

preference(razdalje(tocke))

{'tocka_0': [('tocka_7', 0.2402517120353761),
  ('tocka_1', 0.2905516771703464),
  ('tocka_3', 0.3579857758946461),
  ('tocka_6', 0.37195258556742417),
  ('tocka_5', 0.38529936543008064),
  ('tocka_4', 0.5518846624937219),
  ('tocka_2', 0.9887242124515765)],
 'tocka_1': [('tocka_0', 0.2905516771703464),
  ('tocka_5', 0.41855517690009236),
  ('tocka_7', 0.45431865641706104),
  ('tocka_3', 0.5310166267425541),
  ('tocka_6', 0.5369340410602236),
  ('tocka_4', 0.5961841658234159),
  ('tocka_2', 0.8772262382281591)],
 'tocka_2': [('tocka_4', 0.5383548127710405),
  ('tocka_5', 0.6264477444540302),
  ('tocka_6', 0.7912910188180975),
  ('tocka_3', 0.8071850596825492),
  ('tocka_1', 0.8772262382281591),
  ('tocka_7', 0.8852056615841982),
  ('tocka_0', 0.9887242124515765)],
 'tocka_3': [('tocka_6', 0.018235533206970606),
  ('tocka_7', 0.12450340126654387),
  ('tocka_5', 0.22146204310594092),
  ('tocka_4', 0.27582236954017064),
  ('tocka_0', 0.3579857758946461),
  ('tocka_1', 0.5310166267425541),

Algoritem je sestavljen iz dveh funkcij:
1) funkcija "Najkrajsa"  pregleda razdalje med točkami. Ko najde najmanjšo razdaljo izmed vseh, izbere tisti dve točki, med katerima ta razdalja nastopi in ju poveže v par. Na koncu nam vrne par točk, s pripadajočo najmanjšo razdaljo.

In [5]:
def Najkrajsa(slovar):
    min_razdalja = sqrt(2)
    par = []
    for key1 in slovar.keys():
        #nastavimo min_razdaljo na najmanjso ki nastopa v slovarju
        if slovar[key1][0][1] <= min_razdalja:
            min_razdalja = slovar[key1][0][1]

    for key2 in slovar.keys():
        if slovar[key2][0][1] == min_razdalja:
            par = [slovar[key2][0][0], key2]
            break

    return(par, min_razdalja)


Najkrajsa(preference(razdalje(tocke)))

(['tocka_6', 'tocka_3'], 0.018235533206970606)

2) funkcija "Vsi_pari" nam omogoči, da se čez 1. funckijo zapeljemo tolikokrat kolikokrat je potrebno in povežemo vse točke v pare. Na koncu nam vrne seznam vseh parov.

In [6]:
def Vsi_pari(slovar):
    pari = []
    for i in range(len(slovar)/2):
        (par, najkrajsa) = Najkrajsa(slovar)
        for e in par:
            # Izbrišemo točki v paru iz slovarja
            if str(e) in slovar:
                del slovar[str(e)]
            # izbrišemo to preferenco še vsem točkam v slovarju
            for elm in slovar.keys():
                #print(elm, slovar[elm])
                for tocka in slovar[elm]:
                    if e in tocka:
                        slovar[elm].remove(tocka)

        pari.append(par)
    return(pari)

#Vsi_pari(preference(razdalje(tocke)))

#Vsi_pari(preference(razdalje(primer_1)))
#Vsi_pari(preference(razdalje(primer_2)))
#Vsi_pari(preference(razdalje(primer_3)))


S funkcijo "vsota_razdalj" izračunamo seštevek vseh razdalj med točkami, ki so skupaj v paru:

In [7]:
def vsota_razdalj(slovar, pari):
    vsota = 0
    # Pogledamo vse pare v "pari", poiščemo točko ki nastopa v slovarju in vzamemo dolžino iz njenih preferenc do točke, s katero je v paru.
    for e in pari:
        for i in range(len(slovar[e[0]])):
            if slovar[e[0]][i][0] == e[1]:
                vsota += slovar[e[0]][i][1]
    return(vsota)

vsota_razdalj(preference(razdalje(tocke)),Vsi_pari(preference(razdalje(tocke))))

1.3170924043628671

Na koncu pa še s pomočjo sledeče funkcije analiziramo, kako se obnaša naša vsota razdalj, ko povečujemo število točk.

In [22]:
def razlicne_vsote(od, do, korak):
    slovar_vsote={}
    for i in range(od, do + 1, korak):
        slovar_vsote[i]=  vsota_razdalj(preference(razdalje(kvadrat_tock(i))),Vsi_pari(preference(razdalje(kvadrat_tock(i)))))
    return(slovar_vsote)


In [21]:
def povprecje(zacetek, konec, korak, ponovitve):
    povprecje = {}
    if ponovitve == 0:
        return povprecje
    base_slovar = razlicne_vsote(zacetek, konec, korak)
    
    """zgeneriramo slovar tolikokrat kolikotr hocenmo ponovitev - 1 (base_slovar je prva ponovitev), 
    vsote pristecvamo v base slovar in koncen rezultat delimo s
    stevilom ponovitev da dobimo povprecje"""
    
    for x in range(ponovitve - 1):
        slovar = razlicne_vsote(zacetek, konec, korak)
        for key, value in slovar.items():
            base_slovar[key] += value
    for key, value in base_slovar.items():
        povprecje[key] = value / ponovitve
    return povprecje

In [23]:
print(povprecje(2, 100 ,10, 5))
plt.plot(*zip(*sorted(povprecje(2, 200 ,20, 20).items())))

#plt.plot(*zip(*sorted(slovar.items())))
plt.show()